In [1]:
import ee

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
# Import datasets
google_ndvi = ee.ImageCollection("MODIS/MOD09GA_006_NDVI")
modis_ndvi = ee.ImageCollection("MODIS/061/MOD13Q1")
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
lhz = ee.FeatureCollection("projects/awatford-bauchlab/assets/KE_LHZ_2011")

In [3]:
REGION_TAGS = ['KE05', 'KE06', 'KE09', 'KE16', 'KE24']
SCALE = 250
MODIS_NDVI_SCALE = 10000
MAX_PIXELS = 100000000

In [12]:
def process_ndvi_image(image, region, scale, max_pixels):
    date = image.get('system:time_start')
    region_mask =  ee.Image.constant(1).clip(region)
    county_bounds = region.geometry().bounds()
    mask = region_mask
    masked_image = image.updateMask(mask)
    value = masked_image.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=county_bounds,
        scale=scale,
        maxPixels=max_pixels).get('NDVI')
    feature = ee.Feature(None, {'date': ee.Date(date).format('MMM dd, YYYY'),
                                'value': value})
    return feature

def process_chirps_image(image, region, scale, max_pixels):
    date = image.get('system:time_start')
    region_mask =  ee.Image.constant(1).clip(region)
    county_bounds = region.geometry().bounds()
    mask = region_mask
    masked_image = image.updateMask(mask)
    value = masked_image.select('precipitation').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=county_bounds,
        scale=scale,
        maxPixels=max_pixels).get('precipitation')
    feature = ee.Feature(None, {'date': ee.Date(date).format('MMM dd, YYYY'),
                                'value': value})
    return feature

def process_modis_ndvi_image(image, region, scale, ndvi_scale, max_pixels):
    date = image.get('system:time_start')
    qa_mask = image.select('SummaryQA')
    region_mask =  ee.Image.constant(1).clip(region)
    region_bounds = region.geometry().bounds()
    mask = qa_mask.eq(0).And(region_mask)
    masked_image = image.updateMask(mask)
    value = masked_image.select('NDVI').divide(ndvi_scale).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region_bounds,
        scale=scale,
        maxPixels=max_pixels).get('NDVI')
    feature = ee.Feature(None, {'date': ee.Date(date).format('MMM dd, YYYY'),
                                'value': value})
    return feature

In [13]:
task_list = []
for region_tag in REGION_TAGS:
    region = lhz.filter(ee.Filter.eq('LZCODE', region_tag))

    ndvi_features = modis_ndvi.map(lambda img: process_modis_ndvi_image(img, region, SCALE, MODIS_NDVI_SCALE, MAX_PIXELS))
    ndvi_task = ee.batch.Export.table.toDrive(collection=ndvi_features, 
                                              description=f'ndvi_{region_tag}', 
                                              folder=region_tag, 
                                              selectors = 'date, value')
    ndvi_task.start()
    task_list.append(ndvi_task)

In [17]:
task_list = []
for region_tag in REGION_TAGS:
    region = lhz.filter(ee.Filter.eq('LZCODE', region_tag))

    ndvi_features = google_ndvi.map(lambda img: process_ndvi_image(img, region, SCALE, MAX_PIXELS))
    ndvi_task = ee.batch.Export.table.toDrive(collection=ndvi_features, 
                                              description=f'ndvi_{region_tag}', 
                                              folder=region_tag, 
                                              selectors = 'date, value')
    ndvi_task.start()
    task_list.append(ndvi_task)

    rainfall_features = chirps.map(lambda img: process_chirps_image(img, region, SCALE, MAX_PIXELS))
    rainfall_task = ee.batch.Export.table.toDrive(collection=rainfall_features, 
                                                  description=f'rainfall_{region_tag}', 
                                                  folder=region_tag, 
                                                  selectors = 'date, value')
    rainfall_task.start()
    task_list.append(rainfall_task)

In [15]:
for task in task_list:
    status = task.status()
    description = status['description']
    state = status['state']
    if state != 'COMPLETED':
        print(state, description)